In [2]:
import cv2
import numpy as np
import os

# SIFT

In [ ]:
img = cv2.imread('DEMOIMGS/holi.jpg')
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
sift = cv2.SIFT_create()
kp = sift.detect(gray,None)
img=cv2.drawKeypoints(gray,kp,img)
cv2.imshow('sift_keypoints',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#draw a circle with size of keypoint and it will even show its orientation
#img=cv2.drawKeypoints(gray,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#cv2.imshow('sift_keypoints.jpg',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
#sift = cv2.SIFT_create()
#kp, des = sift.compute(gray,None)

# Fingerprint match using SIFT 

In [19]:
inp_fp = cv2.imread("FPDB/100__M_Left_index_finger.bmp") #read the fingerprint input image
cv2.imshow("Fingerprint Input", cv2.resize(inp_fp, None, fx=3, fy=3)) #show image in a different window
cv2.waitKey(3000) #wait for close on output window
cv2.destroyAllWindows() #after closing,delete the window

In [17]:
def fp_match(fpin):
    for file in [file for file in os.listdir("FPDB")]: #traverse through the database for matching fingerprint
        fingerprint_database_image = cv2.imread('FPDB/'+file) #reading the traversed file
        #SIFT-->SCALE INVARIANT FEATURE TRANSFORM
        #before SIFT,we need to create a SIFT feature detector object

        #sift = cv2.xfeatures2d.SIFT_create()  for OpenCV 3.4.1
        sift=cv2.SIFT_create()  #for OpenCV 2
                                                                            #FINDING KEYPOINTS(Detect) AND DESCRIPTORS(Compute)
        keypoints_1, descriptors_1 = sift.detectAndCompute(fpin, None) #in the original image 

        keypoints_2, descriptors_2 = sift.detectAndCompute(fingerprint_database_image, None) #in the traversed image

        #Matching and comparing both images using the Keypoints
        #FLANN-Fast Library for Approximate Nearest Neighbors Based Matcher 
        #is used for the comparision of the both,which matches features of both finger prints
        #quick and efficient
        #uses Clustering and Search in MultiDimensional Spaces

        #knnMatch to implement Image matching and sort the match results
        #k=2-->best 2 matches for each keypoint
        #matches = cv2.FlannBasedMatcher(dict(algorithm=FLANN_INDEX_KDTREE, trees=5),dict()).knnMatch(descriptors_1, descriptors_2, k=2)

        matches = cv2.FlannBasedMatcher(dict(algorithm=1, trees=10),dict()).knnMatch(descriptors_1, descriptors_2, k=2)
        match_points = []
        #print(matches)
        for p, q in matches:
            #CHECKING FOR GOOD MATCHES
            if p.distance < 0.1*q.distance:
                match_points.append(p)
            keypoints = 0
            if len(keypoints_1) <= len(keypoints_2):
                keypoints = len(keypoints_1)            
            else:
                keypoints = len(keypoints_2)

            #print(keypoints)

            #if we have a proper match,show the output of matched file,%match and the matching lines
            if (len(match_points) / keypoints)>0.95:
                matchper=round(len(match_points) / keypoints * 100,2) #Successful matches from the features tracked
                matchfromdb=str(file) 
                #result = cv2.drawMatches(test_original, keypoints_1, fingerprint_database_image, keypoints_2, match_points, None) 
                #result = cv2.resize(result, None, fx=5.5, fy=5.5)
                #cv2.imshow("result", result)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                return matchper,matchfromdb
            
matchacc,fname=fp_match(test_original)
print(matchacc,"%")
print(fname)

97.22 %
100__M_Left_index_finger.bmp
